In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from joblib import dump


In [2]:
random_state=42
best_model = {}

In [3]:
# Read in the data
# Read in the data
df = pd.read_csv('../Data/hashed_combined.csv')
df.drop(['fitspatrick','region_hash_0','region_hash_1','region_hash_2','region_hash_3','region_hash_4','region_hash_5','region_hash_6','region_hash_7','region_hash_8','region_hash_9','changed','bleed','elevation','biopsed','diameter_2','diameter_1','itch','grew','hurt'], axis=1, inplace=True)
df

,smoke,drink,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,diagnostic,...,background_mother_hash_0,background_mother_hash_1,background_mother_hash_2,background_mother_hash_3,background_mother_hash_4,background_mother_hash_5,background_mother_hash_6,background_mother_hash_7,background_mother_hash_8,background_mother_hash_9
0,False,False,55,False,0,True,True,True,True,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,False,True,79,False,1,True,False,False,False,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,False,True,52,False,0,False,True,True,True,1,...,0.0,-2.0,1.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0
3,False,False,74,True,0,False,False,False,False,1,...,1.0,-1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,False,True,58,True,0,True,True,True,True,1,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,False,False,23,True,0,False,True,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1701,False,False,27,False,0,False,False,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1702,True,True,23,False,1,False,False,True,True,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1703,False,False,23,True,0,False,False,True,False,0,...,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   smoke                     1705 non-null   bool   
 1   drink                     1705 non-null   bool   
 2   age                       1705 non-null   int64  
 3   pesticide                 1705 non-null   bool   
 4   gender                    1705 non-null   int64  
 5   skin_cancer_history       1705 non-null   bool   
 6   cancer_history            1705 non-null   bool   
 7   has_piped_water           1705 non-null   bool   
 8   has_sewage_system         1705 non-null   bool   
 9   diagnostic                1705 non-null   int64  
 10  background_father_hash_0  1705 non-null   float64
 11  background_father_hash_1  1705 non-null   float64
 12  background_father_hash_2  1705 non-null   float64
 13  background_father_hash_3  1705 non-null   float64
 14  backgrou

In [5]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['diagnostic'], axis=1)
    y = df['diagnostic']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=random_state)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=random_state)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=random_state)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=random_state)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=random_state)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [6]:
def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    DT = DecisionTreeClassifier(random_state=random_state)
    # Fit the classifier to the data
    DT.fit(X_train, y_train)
    return DT

In [7]:
def best_model(modelName, accuracy, precision, recall, f1):
    best_model[modelName] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [8]:
def predict(modleName, DT, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = DT.predict(X_test)
    # accuracy = accuracy_score(y_test, y_pred)
    # cr=classification_report(y_test, y_pred, output_dict=True)
    # precision = cr['weighted avg']['precision']
    # recall = cr['weighted avg']['recall']
    # f1 = cr['weighted avg']['f1-score']
    # best_model(modleName,accuracy,precision,recall,f1)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [9]:




from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_with_grid(X_train, y_train):
    # Define a pipeline that first scales the data and then applies the classifier
    pipe = Pipeline([
        # ('scaler', StandardScaler()),
        ('dt', DecisionTreeClassifier(random_state=random_state))
    ])

    # Define the parameter grid to search
    param_grid = {
        'dt__max_depth': [None, 10, 20, 30, 40, 50],
        'dt__min_samples_split': [2, 5, 10],
        'dt__min_samples_leaf': [1, 2, 4],
        'dt__criterion': ['gini', 'entropy']
    }

    # Create the GridSearchCV object
    DT_cv = GridSearchCV(pipe,param_grid, cv=5, verbose=1, n_jobs=-1)

    # Perform the grid search on the provided data
    DT_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = DT_cv.best_params_
    best_score = DT_cv.best_score_
    best_estimator = DT_cv.best_estimator_
    print(best_params)
    print(best_score)

    return best_estimator


<h1> DT on original data with optimization </h1>

In [10]:
# using function with no sampling 
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0     211
Name: diagnostic, dtype: int64


In [12]:
DT1 =training(X_train, y_train)
y_pred = predict('original',DT1, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



In [13]:
best_DT1 = optimize_with_grid(X_train, y_train)
prediction = predict('original_grid',best_DT1, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 4, 'dt__min_samples_split': 2}
0.9978021978021978
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



<h1> DT using SMOTE sampling </h1>

In [14]:
X,y = splitting_data(df, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [16]:
DT2 =training(X_train, y_train)
y_pred = predict('SMOTE',DT2, X_test, y_test)

# # Assume 'model' is your trained model
# dump(DT2, '../Models/DT_SMOTE.joblib')


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       284
           1       1.00      0.99      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [17]:
best_DT2 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTE_grid',best_DT2, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 2}
0.999163179916318
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       284
           1       1.00      0.99      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> DT using SMOTEENN sampling </h1>

In [18]:
X,y = splitting_data(df, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    1493
1    1481
Name: diagnostic, dtype: int64


In [20]:
DT3 =training(X_train, y_train)
y_pred = predict('SMOTEENN',DT3, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       304
           1       1.00      1.00      1.00       291

    accuracy                           1.00       595
   macro avg       1.00      1.00      1.00       595
weighted avg       1.00      1.00      1.00       595



In [21]:
best_DT3 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTEENN_grid',best_DT3, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.9991596638655462
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       304
           1       1.00      1.00      1.00       291

    accuracy                           1.00       595
   macro avg       1.00      1.00      1.00       595
weighted avg       1.00      1.00      1.00       595



<h1> DT on Random undersampling </h1>

In [22]:
X,y = splitting_data(df, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [24]:
DT4 =training(X_train, y_train)
y_pred = predict('undersampling',DT4, X_test, y_test)

              precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       1.00      0.98      0.99        50

    accuracy                           0.99        85
   macro avg       0.99      0.99      0.99        85
weighted avg       0.99      0.99      0.99        85



In [25]:
best_DT4 = optimize_with_grid(X_train, y_train)
prediction = predict('undersampling_grid',best_DT4, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.9940298507462686
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       1.00      0.98      0.99        50

    accuracy                           0.99        85
   macro avg       0.99      0.99      0.99        85
weighted avg       0.99      0.99      0.99        85



<h1> DT on Random Oversampling </h1>

In [26]:
X,y = splitting_data(df,'over')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [28]:
DT5 =training(X_train, y_train)
y_pred = predict('oversampling',DT5, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [29]:
best_DT5 = optimize_with_grid(X_train, y_train)
prediction = predict('oversampling_grid',best_DT5, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.999581589958159
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> DT on Cluster Centroids </h1>

In [30]:
X,y = splitting_data(df, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [32]:
DT6 = training(X_train, y_train)
y_pred = predict('cluster_centroids',DT6, X_test, y_test)

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        35
           1       1.00      0.94      0.97        50

    accuracy                           0.96        85
   macro avg       0.96      0.97      0.96        85
weighted avg       0.97      0.96      0.96        85



In [33]:
best_DT6 = optimize_with_grid(X_train, y_train)
prediction = predict('cluster_centroids_grid',best_DT6, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 10}
0.9940298507462686
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



<h1> DT on Tomek Links </h1>

In [34]:
X,y = splitting_data(df, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    1493
0     211
Name: diagnostic, dtype: int64


In [36]:
DT7 =training(X_train, y_train)
y_pred = predict('tomek_links',DT7, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       1.00      1.00      1.00       298

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



In [37]:
best_DT7 = optimize_with_grid(X_train, y_train)
prediction = predict('tomek_links_grid',best_DT7, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.9985321051497522
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       1.00      1.00      1.00       298

    accuracy                           1.00       341
   macro avg       1.00      0.99      0.99       341
weighted avg       1.00      1.00      1.00       341



In [38]:
best_model

<function __main__.best_model(modelName, accuracy, precision, recall, f1)>

In [39]:
best_model_df = pd.DataFrame.from_dict(best_model, orient='index')
best_model_df.sort_values(by='accuracy', ascending=False, inplace=True)
best_model_df

TypeError: object of type 'function' has no len()